In [1]:
import torch
from mmcv import load, dump
from pyskl.smp import *
from pyskl.models import build_model
from time import time
from tqdm import tqdm

In [2]:
layout = 'nturgb+d'
graph = dict(layout=layout, mode='spatial')
graph_rdm = dict(layout=layout, mode='random', num_filter=8, init_off=.04, init_std=.02)
graph_bin = dict(layout=layout, mode='binary_adj')
aagcn_cfg = dict(type='AAGCN', graph_cfg=graph)
ctrgcn_cfg = dict(type='CTRGCN', graph_cfg=graph)
dgstgcn_cfg = dict(type='DGSTGCN', gcn_ratio=0.125, gcn_ctr='T', gcn_ada='T', tcn_ms_cfg=[(3, 1), (3, 2), (3, 3), (3, 4), ('max', 3), '1x1'], graph_cfg=graph_rdm)
msg3d_cfg = dict(type='MSG3D', graph_cfg=graph_bin)
stgcn_cfg = dict(type='STGCN', graph_cfg=graph)
stgcnpp_cfg = dict(type='STGCN', gcn_adaptive='init', gcn_with_res=True, tcn_type='mstcn', graph_cfg=graph)

In [3]:
cfg_map = dict(aagcn=aagcn_cfg, ctrgcn=ctrgcn_cfg, dgstgcn=dgstgcn_cfg, msg3d=msg3d_cfg, stgcn=stgcn_cfg, stgcnpp=stgcnpp_cfg)

In [13]:
batch = 16
warmup = 10
iters = 10
num_joints = {'nturgb+d': 25, 'coco': 17, 'openpose': 18}[layout]
num_person = 2
seq_len = 100

- Inference speed tested with above settings (FPS -> sample / second)

| Device / FPS | AAGCN | CTRGCN | DGSTGCN | MSG3D | STGCN | STGCN++ | PoseC3D | Notes |
|--|--|--|--|--|--|--|--|--|
| 2080Ti (Linux) | 274 | 353 | 409 | 111 | 518 | 476 | 41 | Iters 100, Warmup 100 |
| 3060 (Windows) | 135 | 148 | 181 | 50 | 206 | 186 | 38 | Iters 100, Warmup 100 |
| CPU (11800H Windows) | 7.5 | 7.2 | 8.1 | 4.3 | 13.8 | 8.9 | 2.9 | Iters 10, Warmup 10 |

In [14]:
# Measure FPS
for k, v in cfg_map.items():
    gcn = build_model(v)
    gcn.eval()
    start = 0
    for i in range(warmup + iters):
        if i == warmup:
            start = time()
        inp = torch.randn(batch, num_person, seq_len, num_joints, 3)
        with torch.no_grad():
            out = gcn(inp)
    end = time()
    fps = (batch * iters) / (end - start)
    print(f'{k} FPS: {fps}')

aagcn FPS: 7.512336739640367
ctrgcn FPS: 7.161762537141133
dgstgcn FPS: 8.095086634695258
msg3d FPS: 4.3017158277906375
stgcn FPS: 13.779976396437187
stgcnpp FPS: 8.861014139175074


In [17]:
posec3d_cfg=dict(
    type='ResNet3dSlowOnly',
    in_channels=17,
    base_channels=32,
    num_stages=3,
    out_indices=(2, ),
    stage_blocks=(4, 6, 3),
    conv1_stride=(1, 1),
    pool1_stride=(1, 1),
    inflate=(0, 1, 1),
    spatial_strides=(2, 2, 2),
    temporal_strides=(1, 1, 2))

In [21]:
batch = 8
warmup = 10
iters = 10
num_joints = 17
seq_len = 48
size = 56

In [22]:
posec3d = build_model(posec3d_cfg)
posec3d.eval()
start = 0
for i in range(warmup + iters):
    if i == warmup:
        start = time()
    inp = torch.randn(batch, num_joints, seq_len, size, size)
    with torch.no_grad():
        out = posec3d(inp)
end = time()
fps = (batch * iters) / (end - start)
print(f'PoseC3D FPS: {fps}')

PoseC3D FPS: 2.8657506130444377
